In [1]:
import mysql.connector as con
import pandas as pd
import random
import datetime 
from datetime import datetime, timedelta


## ENUNCIADO EJERCICIO 

### 1. CREAR BASE DE DATOS (33%)

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.

In [2]:
def crear_tabla(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

In [3]:
sql = "DROP DATABASE IF EXISTS supermercado; CREATE DATABASE IF NOT EXISTS supermercado;"

In [4]:
crear_tabla(sql)

## Crear TABLAS

tiendas
- id_tienda (PRIMARY KEY)
- nombre_tienda
- direccion
- ciudad

In [5]:
sql = "CREATE TABLE IF NOT EXISTS tiendas (id_tienda INT AUTO_INCREMENT PRIMARY KEY, nombre_tienda VARCHAR(255) NOT NULL, direccion VARCHAR(255), ciudad VARCHAR(255) NOT NULL);"

In [6]:
crear_tabla(sql)

empleados
- id_empleado (PRIMARY KEY)
- nombre_empleado
- puesto (ej.: Cajero, Gerente, Reponedor)
- id_tienda (FOREIGN KEY que hace referencia a tiendas.id_tienda)

In [ ]:
sql = "CREATE TABLE IF NOT EXISTS empleados (id_empleado INT AUTO_INCREMENT PRIMARY KEY, nombre_empleado VARCHAR(255) NOT NULL, puesto VARCHAR(60), id_tienda INT NOT NULL, FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE);"

In [8]:
crear_tabla(sql)

categorias
- id_categoria (PRIMARY KEY)
- nombre_categoria

In [9]:
sql = "CREATE TABLE IF NOT EXISTS categorias (id_categoria INT AUTO_INCREMENT PRIMARY KEY, nombre_categoria VARCHAR(60));"

In [10]:
crear_tabla(sql)

productos
- id_producto (PRIMARY KEY)
- nombre_producto
- precio
- stock
- id_categoria (FOREIGN KEY que hace referencia a categorias.id_categoria)

In [11]:
sql = "CREATE TABLE IF NOT EXISTS productos (id_producto INT AUTO_INCREMENT PRIMARY KEY, nombre_producto VARCHAR(70), precio FLOAT(8,2), stock INT, id_categoria INT NOT NULL, FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria));"

In [12]:
crear_tabla(sql)

clientes
- id_cliente (PRIMARY KEY)
- first_name
- last_name
- email
- codigo_postal

In [13]:
sql = "CREATE TABLE IF NOT EXISTS clientes (id_cliente INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), email VARCHAR(200), codigo_postal INT);"

In [14]:
crear_tabla(sql)

ordenes
- id_orden (PRIMARY KEY)
- id_cliente (FOREIGN KEY que hace referencia a clientes.id_cliente)
- id_empleado (FOREIGN KEY que hace referencia a empleados.id_empleado)
- fecha_orden
- metodo_pago (una enum que solo admita tres valores Tarjeta, Efectivo)

In [15]:
sql = "CREATE TABLE IF NOT EXISTS ordenes (id_orden INT AUTO_INCREMENT PRIMARY KEY, id_cliente INT, id_empleado INT, fecha_orden DATE, metodo_pago ENUM('Tarjeta', 'Efectivo') DEFAULT 'Efectivo', FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente), FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado));"

In [16]:
crear_tabla(sql)

detalle_orden
- id_detalle (PRIMARY KEY)
- id_orden (FOREIGN KEY que hace referencia a ordenes.id_orden) NOT NULL
- id_producto (FOREIGN KEY que hace referencia a productos.id_producto) NOT NULL
- cantidad
- precio_unitario: mismo precio que en la tabla producto
- descuento (podría ser NULL si no se aplica)

In [17]:
sql = "CREATE TABLE IF NOT EXISTS detalle_orden (id_detalle INT AUTO_INCREMENT PRIMARY KEY, id_orden INT NOT NULL, id_producto INT NOT NULL, cantidad FLOAT(8,2), precio_unitario FLOAT(8,2), descuento TINYINT, FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden), FOREIGN KEY (id_producto) REFERENCES productos(id_producto));"

In [18]:
crear_tabla(sql)

### 2. Generar datos demo desde Python (33%)

* Generar datos aleatorios en listas con Python similar los realizados en clase.
    * Uso de datetime, timedelta, random para generar datos aleatorios
* Pasar los datos a DataFrames de Pandas
* Pasar los DataFrames de Pandas a MySQL usando la función to_sql de Pandas con SQLAlchemy o usando MySQL Connector con sentencias INSERT.

In [19]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] # lista de tuplas con los datos del dataframe

        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

tiendas

* id_tienda: valores enteros consecutivos (por ejemplo, 1, 2, 3...).
* nombre_tienda: nombres ficticios o genéricos (p. ej. "Tienda Centro", "Super Norte", "Super Sur").
* direccion: direcciones simples (p. ej. “Calle Falsa 123”).
* ciudad: usar nombres de ciudades ficticias o reales (p. ej. “Madrid”, “Barcelona”, “México DF”, etc.).

En total 5 o 10 tiendas.

In [ ]:
tiendas = ['Jerez Este', 'Sanlucar Oeste', 'Cadiz Norte', 'Valdelagrana Sur', 'Arcos Sur', 'Chiclana Norte', 'Barbate Este', 'Vejer Oeste', 'Conil Sur', 'Rota Norte']
direcciones = ['Calle Catavino 25', 'Calle Manzanilla 42', 'Calle del Pan 38', 'Calle Playa 57', 'Calle Castillo 182', 'Calle Barrosa 63', 'Calle Atun 98', 'Calle Pescadill 56', 'Calle Balsa 71', 'Calle Base 22']
ciudades = ['Jerez de la Frontera', 'Sanlucar de Barrameda', 'Cadiz', 'El Puerto de Santa Maria', 'Arcos de la Frontera', 'Chiclana de la Frontera', 'Barbate', 'Vejer de la Frontera', 'Conil de la Frontera', 'Rota']
df_tiendas = pd.DataFrame(zip(tiendas, direcciones, ciudades), columns=['nombre_tienda', 'direccion', 'ciudad'])
# generar columna id_tienda
df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas

In [ ]:
insertar_dataframe(df_tiendas, 'tiendas')

empleados

* id_empleado: valores enteros consecutivos (1, 2, 3...).
* nombre_empleado: nombres y apellidos ficticios (p. ej. “Laura Gutiérrez”, “Juan Pérez”).
* puesto: limitarse a un conjunto reducido de valores (p. ej. {‘Cajero’, ‘Gerente’, ‘Reponedor’, ‘Vendedor’}).
* id_tienda: debe hacer referencia a una tienda existente (por ejemplo, un número entre 1 y 3 si solo tienes 3 tiendas).

En total 20 empleados por tienda.

In [ ]:
#generar columna id_empleado
empleados = list()
nombres = ["Juan", "María", "Luis", "Ana", "Pedro", "Sofía", "Carlos", "Lucía", "Jorge", "Elena", "Manuel", "Laura", "Diego", "Clara", "Alberto", "Teresa", "Pablo", "Isabel", "Sergio", "Marta", "Roberto", "Julia", "Fernando", "Carmen", "Ricardo", "Paula", "Andrés", "Eva", "Antonio", "Sara", "José", "Natalia", "Francisco", "Patricia", "Hugo", "Adriana", "Rafael", "Irene", "Víctor", "Olga", "Daniel", "Noelia", "Álvaro", "Alicia", "Javier", "Rocío", "Enrique", "Amelia", "Gonzalo", "Vera"]
apellidos = ["García", "Martínez", "López", "Sánchez", "Pérez", "González", "Rodríguez", "Fernández", "Moreno", "Jiménez", "Hernández", "Muñoz", "Álvarez", "Romero", "Alonso", "Gutiérrez", "Ruiz", "Navarro", "Díaz", "Torres", "Castro", "Vázquez", "Ramos", "Gil", "Ramírez", "Flores", "Ortiz", "Marín", "Serrano", "Iglesias", "Delgado", "Domínguez", "Guerrero", "Molina", "Ortega", "Rubio", "Santos", "Medina", "Castillo", "Cruz", "Reyes", "Herrera", "Peña", "Carrasco", "Calvo", "Vega", "Cabrera", "Campos", "León", "Fuentes"]
trabajo = ["Cajero", "Reponedor", "Encargado de sección", "Gerente", "Personal de limpieza", "Panadero", "Carnicero", "Pescadero", "Responsable de almacén", "Supervisor de caja", "Frutero", "Charcutero", "Jefe de turno", "Operador logístico", "Atención al cliente", "Auxiliar de caja", "Encargado de inventario", "Responsable de compras", "Promotor de ventas", "Encargado de seguridad"]
for tienda in df_tiendas['id_tienda']:
    for empleado in range(20):
        nombre_empleado = (random.choice(nombres) + ' ' + random.choice(apellidos))
        puesto = trabajo[empleado]
        id_tienda = tienda
        empleados.append([nombre_empleado, puesto, id_tienda])
        
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
        
df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados

In [ ]:
insertar_dataframe(df_empleados, "empleados")

categorias

* id_categoria: valores enteros consecutivos (1, 2, 3...).
* nombre_categoria: selección de categorías (p. ej. “Lácteos”, “Carnes”, “Frutas”, “Verduras”, “Bebidas”, “Snacks”).

En total 10 categorías

In [ ]:
categorias = ['Lácteos', 'Carnes', 'Frutas', 'Verduras', 'Bebidas', 'Snacks', 'Hogar', 'Limpieza', 'Perfumería', 'Jardín']

df_categorias = pd.DataFrame(categorias, columns=['nombre_categoria'])

# generar columna id_categoria
df_categorias = df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias

In [ ]:
insertar_dataframe(df_categorias, 'categorias')


productos

* id_producto: valores enteros consecutivos (1, 2, 3...).
* nombre_producto: nombres como “Leche Entera”, “Manzana Roja”, “Refresco de Cola”, etc.
* precio: valores DECIMAL entre 0.50 y 50.00, por ejemplo.
* stock: valores INT entre 0 y 500 (o el rango que quieras).
* id_categoria: debe hacer referencia a las categorías que hayas definido (1, 2, 3, etc.).

En total 4 productos de cada categoría.

In [46]:
# generar columna id_producto
nombres = ["Leche", "Yogur", "Queso", "Mantequilla", "Pollo", "Carne de res", "Cerdo", "Chuletas", "Manzana", "Banana", "Naranja", "Uva", "Zanahoria", "Tomate", "Lechuga", "Cebolla", "Agua", "Jugo de naranja", "Refresco", "Cerveza", "Papas fritas", "Chocolate", "Galletas", "Chicles", "Velas", "Platos desechables", "Bolsas de basura", "Pilas", "Detergente", "Jabón líquido", "Esponjas", "Desinfectante", "Champú", "Colonia", "Crema hidratante", "Desodorante", "Tierra para macetas", "Fertilizante", "Semillas", "Regadera"]
precios, lista_categorias, stock = list(), list(), list()
cantidad = len(nombres)
for _ in range(cantidad):
    precios.append(random.uniform(0.50, 10.50))
    stock.append(random.randint(0, 500))
for categ in df_categorias['id_categoria']:
    for i in range(4):
        lista_categorias.append(categ)
lista_categorias        
df_productos = pd.DataFrame(zip(nombres, precios, stock, lista_categorias), columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])    
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Leche,4.947031,334,1
1,2,Yogur,10.304934,352,1
2,3,Queso,3.053225,298,1
3,4,Mantequilla,9.262840,250,1
4,5,Pollo,8.669271,198,2
5,6,Carne de res,9.358373,182,2
6,7,Cerdo,7.884616,33,2
7,8,Chuletas,9.495966,337,2
8,9,Manzana,4.526493,166,3
9,10,Banana,4.318121,102,3


In [47]:
insertar_dataframe(df_productos, 'productos')

40

clientes

* id_cliente: valores enteros consecutivos (1, 2, 3...).
* nombre_cliente: nombres y apellidos ficticios (p. ej. “Carlos López”, “María Gil”).
* email: podrías generar correos ficticios (p. ej. “carlos.lopez@test.com”).
* telefono: número de 9 o 10 dígitos (dependiendo del país).
* direccion: calles y números ficticios (p. ej. “Av. Siempre Viva 742”).

En total: 2000.


In [ ]:
# generar columna id_cliente
nombre_cliente, apellido_cliente, email, phone, direccion = list(), list(), list(), list(), list()

df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

In [ ]:
insertar_dataframe(df_clientes, 'clientes')

ordenes

* id_orden: valores enteros consecutivos (1, 2, 3...).
* id_cliente: haz referencia a los IDs existentes de la tabla clientes.
* id_empleado: haz referencia a los IDs existentes de la tabla empleados.
* fecha_orden: genera fechas aleatorias o secuenciales (p. ej. entre ‘2024-01-01’ y ‘2025-01-01’).
* metodo_pago: escoge entre {‘Tarjeta’, ‘Efectivo’}.

En total: 10000.


In [ ]:
# generar columna id_orden
df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes

In [ ]:
insertar_dataframe(df_ordenes, 'ordenes')

detalle_orden

* id_detalle: valores enteros consecutivos (1, 2, 3...).
* id_orden: referencia al ID de alguna orden válida.
* id_producto: referencia al ID de algún producto válido.
* cantidad: valores entre 1 y 20, por ejemplo.
* precio_unitario: usar el mismo precio que está en la tabla productos o uno ligeramente distinto si quieres simular ofertas.
* descuento: valores DECIMAL bajos (p. ej. 0.00, 1.00, 2.50) o NULL.

En total: 30000.

In [ ]:
# generar columna id_detalle
df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1
df_detalle_orden

In [ ]:
insertar_dataframe(df_detalle_orden, 'detalle_orden')

### 3. Consultas SQL (34%)

1. Listado de órdenes con detalles de cliente y empleado
* Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado que atendió la compra y el método de pago.
* Utiliza un JOIN entre las tablas ordenes, clientes y empleados.

2. Productos con stock bajo
* Filtra aquellos productos cuyo stock sea menor a 10.
* Muestra nombre del producto, categoría y stock.


3. Ventas totales por categoría
* Muestra el nombre de la categoría y la suma total de las ventas (ej.: multiplicando cantidad * precio_unitario) para cada categoría.
* Realiza el JOIN con detalle_orden, productos y categorias.
* Utiliza agrupación (GROUP BY).

4. Clientes con mayores gastos acumulados
* Muestra el nombre del cliente y el monto total que ha gastado (suma de todas sus órdenes).
* Asegúrate de tener en cuenta posibles descuentos (descuento) si se ha definido. Por ejemplo, la fórmula podría ser (cantidad * precio_unitario) - descuento.
* Ordena el resultado de mayor a menor gasto acumulado.

5. Empleados y número de órdenes gestionadas
* Muestra el nombre del empleado, el puesto y la cantidad de órdenes que ha gestionado.
* Utiliza GROUP BY y COUNT.

6. Ordenes filtradas por fecha y tienda
* Muestra todas las órdenes que se realizaron en un rango de fechas determinado (ej.: del 1 de enero de 2025 al 31 de enero de 2025) y en una tienda específica.
* Incluye datos de la tienda y del cliente.

7. Ranking de productos más vendidos en cada tienda
* Para cada tienda, muestra los 3 productos más vendidos (en términos de cantidad total).
* Tendrás que unir tiendas, empleados, ordenes y detalle_orden, además de productos.
* Usa GROUP BY y ordena por la cantidad sumada (y opcionalmente, un LIMIT 3).


Opcional: añadir alguna consulta con subconsultas o algo que no se abarque en las anteriores consulta.